In [44]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import re
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from pygam import LinearGAM, s
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from statsmodels.discrete.discrete_model import Probit
from scipy.stats import norm
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz
from xgboost import XGBClassifier
from xgboost import XGBRegressor


In [45]:
ties_df = pd.read_excel("TIESv4-1.xls")
cow_df = pd.read_csv("COW-country-codes.csv")
plty_df = pd.read_excel("POLITY5-PRC.xlsx")
prob_df = pd.read_csv("all_data_probabilities.csv")

In [46]:
# Identify columns that are years
year_columns = [col for col in plty_df.columns if col.isdigit()]

# Melt the entire Polity dataset (no filtering by Indicator)
melted_df_auto = plty_df.melt(
    id_vars=['Economy Name', 'Indicator'], 
    value_vars=year_columns, 
    var_name='Year', 
    value_name='Value'
)

# Convert Year and Value to numeric and drop missing
melted_df_auto['Year'] = pd.to_numeric(melted_df_auto['Year'], errors='coerce')
melted_df_auto = melted_df_auto.dropna(subset=['Year'])
melted_df_auto['Year'] = melted_df_auto['Year'].astype(int)

melted_df_auto['Value'] = pd.to_numeric(melted_df_auto['Value'], errors='coerce')
melted_df_auto = melted_df_auto.dropna(subset=['Value'])

# Standardize country names
def safe_upper_strip(x):
    if isinstance(x, str):
        return x.upper().strip()
    return np.nan

melted_df_auto['Economy_Name_standardized'] = melted_df_auto['Economy Name'].apply(safe_upper_strip)
cow_df['StateNme_standardized'] = cow_df['StateNme'].apply(safe_upper_strip)

melted_df_auto = melted_df_auto.dropna(subset=['Economy_Name_standardized'])
cow_df = cow_df.dropna(subset=['StateNme_standardized'])

# Map countries to their CCode
country_mapping = dict(zip(cow_df['StateNme_standardized'], cow_df['CCode']))
melted_df_auto['Country_Code'] = melted_df_auto['Economy_Name_standardized'].map(country_mapping)
melted_df_auto = melted_df_auto.dropna(subset=['Country_Code'])
melted_df_auto['Country_Code'] = melted_df_auto['Country_Code'].astype(int)

melted_df_auto = melted_df_auto[~(
    (melted_df_auto['Indicator'] == 'Polity database: Combined Polity Score') &
    ((melted_df_auto['Value'] < -10) | (melted_df_auto['Value'] > 10))
)]

# Ensure ties_df has numeric Country_Code and year
ties_df['Country_Code'] = pd.to_numeric(ties_df['targetstate'], errors='coerce')
ties_df = ties_df.dropna(subset=['Country_Code'])
ties_df['Country_Code'] = ties_df['Country_Code'].astype(int)

ties_df['startyear'] = pd.to_numeric(ties_df['startyear'], errors='coerce')
ties_df = ties_df.dropna(subset=['startyear'])
ties_df['startyear'] = ties_df['startyear'].astype(int)

# Pivot to get a multi-level column structure: columns = Indicator, Year
polity_wide = melted_df_auto.pivot_table(
    index='Country_Code', 
    columns=['Indicator','Year'], 
    values='Value'
)

# Define year offsets
year_offsets = range(-20, 21)

# For each row in ties_df, we extract polity scores for all indicators and all offsets
def get_polity_scores_for_row(row):
    ccode = row['Country_Code']
    event_year = row['startyear']
    if ccode not in polity_wide.index:
        # No polity data for this country
        return [np.nan] * (len(polity_wide.columns.levels[0]) * len(year_offsets))
    
    country_data = polity_wide.loc[ccode]
    all_values = []
    
    # Loop over each indicator
    for indicator in polity_wide.columns.levels[0]:
        # Get just this indicator's time series
        indicator_series = country_data[indicator]
        years_to_extract = [event_year + off for off in year_offsets]
        extracted = indicator_series.reindex(years_to_extract)
        all_values.extend(extracted.values)
    
    return all_values

# Create column names for all indicators and offsets
indicator_names = polity_wide.columns.levels[0]
all_columns = []
for ind in indicator_names:
    for offset in year_offsets:
        all_columns.append(f"{ind}_{offset}")

polity_scores = ties_df.apply(get_polity_scores_for_row, axis=1)
polity_scores_df = pd.DataFrame(polity_scores.tolist(), columns=all_columns, index=ties_df.index)

# Concatenate these columns with the original ties_df
final_df = pd.concat([ties_df, polity_scores_df], axis=1)

In [47]:
final_df

,caseid,startmonth,startday,startyear,endmonth,endday,endyear,ongoingasofmonth,ongoingasofday,ongoingasofyear,...,Polity database: The Competitiveness of Participation_11,Polity database: The Competitiveness of Participation_12,Polity database: The Competitiveness of Participation_13,Polity database: The Competitiveness of Participation_14,Polity database: The Competitiveness of Participation_15,Polity database: The Competitiveness of Participation_16,Polity database: The Competitiveness of Participation_17,Polity database: The Competitiveness of Participation_18,Polity database: The Competitiveness of Participation_19,Polity database: The Competitiveness of Participation_20
0,1945121601,12.0,16.0,1945,5.0,27.0,1947.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1946020801,2.0,8.0,1946,6.0,19.0,1949.0,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,1946031101,3.0,11.0,1946,10.0,9.0,1993.0,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,1946040901,4.0,9.0,1946,8.0,3.0,1960.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1946051001,5.0,10.0,1946,8.0,9.0,1946.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2005060701,6.0,7.0,2005,NaN,NaN,NaN,8.0,12.0,2011.0,...,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,2005062801,6.0,28.0,2005,9.0,1.0,2005.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,2005071401,7.0,14.0,2005,8.0,2.0,2005.0,NaN,NaN,NaN,...,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,2005100601,10.0,6.0,2005,6.0,17.0,2010.0,NaN,NaN,NaN,...,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Create a mapping from ISO3 (StateAbb) to CCode using cow_df
iso3_to_ccode = dict(zip(cow_df['StateAbb'], cow_df['CCode']))

# Add a column for next year's prediction to prob_df
prob_df['Year_next'] = prob_df['Year'] + 1

# Map ISO3 to Country_Code using the iso3_to_ccode dictionary
prob_df['Country_Code'] = prob_df['ISO3'].map(iso3_to_ccode)

# Drop rows where mapping failed (no ISO3 match in cow_df)
prob_df = prob_df.dropna(subset=['Country_Code'])

# Ensure Country_Code is integer
prob_df['Country_Code'] = prob_df['Country_Code'].astype(int)

# Merge on Country_Code and Year_next to align next-year probabilities with final_df events
final_merged = pd.merge(
    final_df,
    prob_df[['Country_Code', 'Year_next', 'predicted_probability']], 
    left_on=['Country_Code', 'startyear'],
    right_on=['Country_Code', 'Year_next'],
    how='left'
)

# Drop Year_next if not needed
final_merged = final_merged.drop(columns='Year_next')


In [54]:
final_merged

,caseid,startmonth,startday,startyear,endmonth,endday,endyear,ongoingasofmonth,ongoingasofday,ongoingasofyear,...,Polity database: The Competitiveness of Participation_12,Polity database: The Competitiveness of Participation_13,Polity database: The Competitiveness of Participation_14,Polity database: The Competitiveness of Participation_15,Polity database: The Competitiveness of Participation_16,Polity database: The Competitiveness of Participation_17,Polity database: The Competitiveness of Participation_18,Polity database: The Competitiveness of Participation_19,Polity database: The Competitiveness of Participation_20,predicted_probability
0,1945121601,12.0,16.0,1945,5.0,27.0,1947.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1,1946020801,2.0,8.0,1946,6.0,19.0,1949.0,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN
2,1946031101,3.0,11.0,1946,10.0,9.0,1993.0,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN
3,1946040901,4.0,9.0,1946,8.0,3.0,1960.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
4,1946051001,5.0,10.0,1946,8.0,9.0,1946.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2005060701,6.0,7.0,2005,NaN,NaN,NaN,8.0,12.0,2011.0,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011302
1408,2005062801,6.0,28.0,2005,9.0,1.0,2005.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,2005071401,7.0,14.0,2005,8.0,2.0,2005.0,NaN,NaN,NaN,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,2005100601,10.0,6.0,2005,6.0,17.0,2010.0,NaN,NaN,NaN,...,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011302


In [55]:
final_merged = final_merged[final_merged['startyear'] >= 1980]

In [56]:
target = "predicted_probability"

final_merged = final_merged.dropna(subset=[target])

# Define columns to drop that are not features:
# This might include identifiers and columns you don't want as features.
drop_cols = ['ISO3', 'Country_Code', 'startyear', target]

# Create y from the target column
y = final_merged[target].values

# Create X by dropping non-feature columns
X = final_merged.drop(columns=drop_cols, errors='ignore')

In [70]:
# Copy the final_df to a working dataframe
analysis_df = X.copy()

# Check and filter by settlementnaturesender
if 'settlementnaturesender' not in analysis_df.columns:
    raise ValueError("settlementnaturesender column not found.")

analysis_df = analysis_df.dropna(subset=['settlementnaturesender'])
analysis_df = analysis_df[analysis_df['imposition'] != 'Missing']

analysis_df = analysis_df[
    (analysis_df['startmonth'] != analysis_df['sancimpositionstartmonth']) |
    (analysis_df['startday'] != analysis_df['sancimpositionstartday'])
]

In [74]:
# Identify polity -1 year variables
polity_minus_one_vars = [col for col in analysis_df.columns if col.endswith('_-1')]

# Listed TIES features (original + newly added)
ties_features = [
    'issue1', 'threatid1', 'sanctiontypethreat', 'bspecif', 'scommit',
    'threatenedtargetinterest', 'carrots', 'anticipatedtargetcosts',
    'anticipatedsendercosts',
    # Newly added features:
    'institution',  # Binary/ Missing -> treated as categorical
    'targetinstitution',  # Binary / Missing -> categorical
    'othersanctiontypethreatened', # categorical
    'dsanctions' # categorical (diplomatic sanctions)
]

all_features = polity_minus_one_vars + ties_features

# Check which features exist
final_features = []
for f in all_features:
    if f not in analysis_df.columns:
        print(f"Feature '{f}' not found, skipping it.")
    else:
        final_features.append(f)

if not final_features:
    raise ValueError("No features available after checking existence. Please revise variable selection.")

# Attempt to convert identified vars to numeric if possible
vars_to_process = final_features + ['settlementnaturesender']
for var in vars_to_process:
    analysis_df[var] = pd.to_numeric(analysis_df[var], errors='coerce')

analysis_df = analysis_df[final_features]

# Now treat ties_features as categorical
for var in ties_features:
    if var in analysis_df.columns:
        # Convert all entries to string to ensure consistency
        analysis_df[var] = analysis_df[var].astype(str)
        analysis_df[var] = analysis_df[var].astype('category')
        # Add a "Missing" category to handle NaN explicitly
        analysis_df[var] = analysis_df[var].cat.add_categories(["Missing"])
        analysis_df[var] = analysis_df[var].fillna("Missing")

# For polity_minus_one_vars, median-impute missing values
for var in polity_minus_one_vars:
    if var in analysis_df.columns:
        # Already numeric from above step
        col_median = analysis_df[var].median()
        analysis_df[var] = analysis_df[var].fillna(col_median)

# Custom parsing function for any remaining non-numeric columns not in ties_features
def parse_numeric_string(val):
    if pd.isna(val):
        return val
    s = str(val)
    # Try to convert directly
    try:
        return float(s)
    except ValueError:
        # If it fails, maybe it's a space-separated pair of numbers
        parts = s.split()
        if len(parts) == 2:
            try:
                num1, num2 = float(parts[0]), float(parts[1])
                return (num1 + num2)/2.0
            except ValueError:
                return np.nan
        # If no known pattern, return NaN
        return np.nan

non_numeric_cols = analysis_df.select_dtypes(include=['object']).columns
for col in non_numeric_cols:
    if col not in ties_features:
        analysis_df[col] = analysis_df[col].apply(parse_numeric_string)
        # Try converting to numeric again
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Remove columns that are entirely NaN
for var in list(final_features):
    if var not in analysis_df.columns:
        continue
    if analysis_df[var].isna().all():
        print(f"Feature '{var}' is all NaN, removing it.")
        final_features.remove(var)

if not final_features:
    raise ValueError("No usable features remain after removing all-NaN columns.")

# Define numerical and categorical features
numerical_features = [f for f in final_features if f in polity_minus_one_vars]
categorical_features = [f for f in ties_features if f in final_features]

# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=0.90))  # Retain 90% of variance
        ]), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)

# Define the full pipeline with a classifier
pipeline_prob_impose = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))  # Adjust parameters as needed
])

# Prepare target and features
X = analysis_df[numerical_features + categorical_features]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Fit the pipeline on training data
pipeline_prob_impose.fit(X_train, y_train)

# Evaluate the model
train_score = pipeline_prob_impose.score(X_train, y_train)
test_score = pipeline_prob_impose.score(X_test, y_test)

print("Number of observations:", len(analysis_df))
onehot_feature_names = pipeline_prob_impose.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features)
print("Number of features after encoding:", len(numerical_features) + onehot_feature_names.shape[0])
print("Training Accuracy:", train_score)
print("Test Accuracy:", test_score)

# Predict probabilities on the test set
y_pred_proba = pipeline_prob_impose.predict_proba(X_test)[:, 1]

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC AUC on test set:", roc_auc)

# Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0,1], [0,1], 'k--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

C:\Users\maxla\AppData\Local\Temp\ipykernel_51168\1354972182.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_df[var] = pd.to_numeric(analysis_df[var], errors='coerce')


KeyError: 'settlementnaturesender'